In [1]:
import numpy as np
import igraph as ig
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from time import time

from common import *
from generate import *
from filter_kruskal import *

In [2]:
common.run(args="--help")

Allowed options:
  -h [ --help ]                      produce help message
  -m [ --mst-kernel ] arg (=1)       Kernel used calculating MST. 0 = regular 
                                     kruskal, 1 = filter-kruskal
  -s [ --sort-kernel ] arg (=0)      Kernel used for sort() [int]
  -p [ --partition-kernel ] arg (=1) Kernel used for partition() [int]
  -f [ --filter-kernel ] arg (=0)    Kernel used for filter() [int]
  -c [ --compress-kernel ] arg (=0)  Kernel used for compress() [int]
  -i [ --inputfile ] arg             input file containing graph data. Filepath
                                     must be relative to Ex2/
  -t [ --ouputfile_timings ] arg     Output file for timings. Filepath must be 
                                     relative to Ex2/



In [7]:
args = f"--inputfile input_data/barabasi_50000_10.csv --mst-kernel 0"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv", args=args, print_output=True)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv --inputfile input_data/barabasi_50000_10.csv --mst-kernel 0

Benchmark results:
Initialize	µ=0.0631596s	sigma=0s	total=0.0631596	num_calls=1
Kruskal()	µ=12.3379s	sigma=0s	total=12.3379	num_calls=1
grow MST	µ=9.48287s	sigma=0s	total=9.48287	num_calls=1
sort_edgelist()	µ=2.85507s	sigma=0s	total=2.85507	num_calls=1
total	µ=12.4011s	sigma=0s	total=12.4011	num_calls=1


,tag,average,std_dev,total,num_calls
0,Initialize,0.06316,0,0.06316,1
1,Kruskal(),12.33790,0,12.33790,1
2,grow MST,9.48287,0,9.48287,1
3,sort_edgelist(),2.85507,0,2.85507,1
4,total,12.40110,0,12.40110,1


In [9]:
args = f"-i input_data/barabasi_50000_10.csv -m 1 -c 0"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv",args=args, print_output=False)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv -i input_data/barabasi_50000_10.csv -m 1 -c 0


,tag,average,std_dev,total,num_calls
0,Initialize,0.062814,0.000000,0.062814,1
1,Kruskal(),0.027071,0.003229,0.649703,24
2,compress(),0.000206,0.000008,0.004948,24
3,filter(),0.000172,0.000061,0.004467,26
4,grow MST,0.007819,0.001975,0.187651,24
5,partition(),0.000573,0.000116,0.014891,26
6,sort_edgelist(),0.019251,0.001667,0.462026,24
7,total,0.736891,0.000000,0.736891,1


In [10]:
args = f"-i input_data/barabasi_50000_10.csv -m 1 -c 1"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv",args=args, print_output=False)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv -i input_data/barabasi_50000_10.csv -m 1 -c 1


,tag,average,std_dev,total,num_calls
0,Initialize,0.050767,0.000000,0.050767,1
1,Kruskal(),0.025743,0.003231,0.617836,24
2,compress(),0.000920,0.000077,0.022079,24
3,filter(),0.000203,0.000059,0.005282,26
4,grow MST,0.007929,0.002005,0.190288,24
5,partition(),0.000537,0.000107,0.013962,26
6,sort_edgelist(),0.017813,0.001593,0.427525,24
7,total,0.710010,0.000000,0.710010,1


In [22]:
args = f"-i input_data/barabasi_500000_1000.csv -m 1 -c 0"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv",args=args, print_output=False)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv -i input_data/barabasi_500000_1000.csv -m 1 -c 0


,tag,average,std_dev,total,num_calls
0,Initialize,0.065658,0.000000,0.065658,1
1,Kruskal(),0.391227,0.047044,19.170100,49
2,compress(),0.002318,0.000071,0.113601,49
3,filter(),0.000483,0.000140,0.033298,69
4,grow MST,0.308549,0.046129,15.118900,49
5,partition(),0.001276,0.000289,0.088010,69
6,sort_edgelist(),0.082677,0.004649,4.051170,49
7,total,19.473800,0.000000,19.473800,1


In [23]:
args = f"-i input_data/barabasi_500000_1000.csv -m 1 -c 1"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv",args=args, print_output=False)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv -i input_data/barabasi_500000_1000.csv -m 1 -c 1


,tag,average,std_dev,total,num_calls
0,Initialize,0.053783,0.000000,0.053783,1
1,Kruskal(),0.389026,0.047364,19.062300,49
2,compress(),0.024427,0.002017,1.196910,49
3,filter(),0.000721,0.000154,0.049746,69
4,grow MST,0.304140,0.046039,14.902900,49
5,partition(),0.001259,0.000280,0.086898,69
6,sort_edgelist(),0.084883,0.004523,4.159290,49
7,total,20.452800,0.000000,20.452800,1
